In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import re

# Etapa 1: nome dos professores

In [2]:
url = 'http://posfisiologia.ib.usp.br/orientadores.html'
html = requests.get(url)

In [3]:
bs = BeautifulSoup(html.text)

In [4]:
fisio_profs = [x.text for x in bs.find('table',).findAll('span', style ='font-size: 14pt;')]
fisio_profs

['Ana Carolina Inhasz Kiss',
 'Ana Maria de Lauro Castrucci',
 'André Frazão Helene',
 'Cristiéle da Silva Ribeiro',
 'Carlos Arturo Navas Iannini',
 'Débora Rejane Fior Chadi',
 'Fernando Ribeiro Gomes',
 'Gilberto Fernando Xavier',
 'Gisele Akemi',
 'Oda',
 'Gustavo Rohenkohl',
 'José Eduardo P.W. Bicudo',
 'José Guilherme de S. Chaui Berlinck',
 'Lucile Maria Floeter-Winter',
 'Márcio Reis Custódio',
 'Maria Aparecida Visconti',
 'Pedro Augusto C.M. Fernandes',
 'Regina Pekelmann Markus',
 'Renata Guimarães Moreira Whitton',
 'Sandra Márcia Muxel',
 'Silvia Cristina Ribeiro de Souza',
 'Zulma Felisbina da Silva Ferreira']

# Etapa 2: link do google scholar dos professores

In [5]:
fisio_profs_google_scholar = ["+".join([x +' google scholar'][0].split()) for x in fisio_profs]

fisio_profs_google_scholar

['Ana+Carolina+Inhasz+Kiss+google+scholar',
 'Ana+Maria+de+Lauro+Castrucci+google+scholar',
 'André+Frazão+Helene+google+scholar',
 'Cristiéle+da+Silva+Ribeiro+google+scholar',
 'Carlos+Arturo+Navas+Iannini+google+scholar',
 'Débora+Rejane+Fior+Chadi+google+scholar',
 'Fernando+Ribeiro+Gomes+google+scholar',
 'Gilberto+Fernando+Xavier+google+scholar',
 'Gisele+Akemi+google+scholar',
 'Oda+google+scholar',
 'Gustavo+Rohenkohl+google+scholar',
 'José+Eduardo+P.W.+Bicudo+google+scholar',
 'José+Guilherme+de+S.+Chaui+Berlinck+google+scholar',
 'Lucile+Maria+Floeter-Winter+google+scholar',
 'Márcio+Reis+Custódio+google+scholar',
 'Maria+Aparecida+Visconti+google+scholar',
 'Pedro+Augusto+C.M.+Fernandes+google+scholar',
 'Regina+Pekelmann+Markus+google+scholar',
 'Renata+Guimarães+Moreira+Whitton+google+scholar',
 'Sandra+Márcia+Muxel+google+scholar',
 'Silvia+Cristina+Ribeiro+de+Souza+google+scholar',
 'Zulma+Felisbina+da+Silva+Ferreira+google+scholar']

# Etapa 3: acessar a página com publicações de cada professor

In [25]:
print(f'https://www.google.com/#q={fisio_profs_google_scholar[2]}')

https://www.google.com/#q=André+Frazão+Helene+google+scholar


In [144]:
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }

dick = {}
for prof in fisio_profs_google_scholar:
    r = requests.get(f'https://www.google.com/search?q={"+".join(prof.split())}', headers_Get)

    bs2 = BeautifulSoup(r.text)
    try:
        x = bs2.findAll('a', href = re.compile('scholar.google'))[0]['href'][7:]

        x2 = urllib.parse.unquote(x)

        x3 = requests.get(x2, headers_Get)

        x4 = BeautifulSoup(x3.content)

        artigos = []
        citacoes = []
        autores = []
        ano = []

        for link in x4.findAll('td', {"class":"gsc_a_t"}):
            artigos.append(link.find('a').text)
            autores.append(link.find('div',{'class':'gs_gray'}).text)
        for link in x4.findAll('td', {'class': 'gsc_a_c'}):
            citacoes.append(int(link.find('a').text))
        nome_prof = '_'.join(x4.find('div', id = 'gsc_prf_in').text.split())
        for link in x4.findAll('td', {"class":"gsc_a_y"}):
            ano.append(int(link.find('span').text))

        dick[nome_prof] = list(zip(artigos,citacoes,autores,ano))
    except IndexError: pass

list(dick)
                     

['Ana_Carolina_Kiss',
 'Ana_Maria_de_Lauro_Castrucci',
 'André_Frazão_Helene',
 'Cristiéle_da_Silva_Ribeiro',
 'Carlos_Arturo_Navas',
 'Debora_Rejane_Fior_Chadi',
 'Fernando_Ribeiro_Gomes',
 'Gilberto_Fernando_Xavier',
 'Gisele_Akemi_Oda',
 'Yoshihisa_Oda',
 'Gustavo_Rohenkohl',
 'Jose_R_Bicudo',
 'José_Guilherme_Chaui-Berlinck',
 'Lucile_Maria_Floeter_Winter',
 'Marcio_Reis_Custodio',
 'Maria_Aparecida_Visconti',
 'Pedro_Augusto_Carlos_Magno_Fernandes',
 'Hazel_Markus',
 'Renata_Guimarães_Moreira',
 'Cristina_Ribeiro',
 'Zulma_Ferreira']

In [145]:
import pandas as pd

In [146]:
l = []
for x in list(dick):
    
    df = pd.DataFrame(dick[x], columns = ['artigo','citacoes','autores','ano'])
    df['professor'] = x
    l.append(df)
df_profs = pd.concat(l)[['professor','artigo','autores','ano','citacoes']]
df_profs 

,professor,artigo,autores,ano,citacoes
0,Ana_Carolina_Kiss,Animal models for clinical and gestational dia...,"ACI Kiss, PHO Lima, YK Sinzato, M Takaku, MA T...",2009,74
1,Ana_Carolina_Kiss,Neonatally-induced diabetes: lipid profile out...,"YK Sinzato, PHO Lima, KE Campos, ACI Kiss, MVC...",2009,54
2,Ana_Carolina_Kiss,"Maternal-fetal outcome, lipid profile and oxid...","DC Damasceno, ACI Kiss, YK Sinzato, KE De Camp...",2011,37
3,Ana_Carolina_Kiss,Seasonal metabolic changes in a year-round rep...,"ACI Kiss, JE de Carvalho, CA Navas, FR Gomes",2009,23
4,Ana_Carolina_Kiss,Genotoxicity and fetal abnormality in streptoz...,"DC Damasceno, GT Volpato, YK Sinzato, PHO Lima...",2011,21
5,Ana_Carolina_Kiss,Impact of maternal mild hyperglycemia on mater...,"ACI Kiss, B Woodside, LF Felício, J Anselmo-Fr...",2012,18
6,Ana_Carolina_Kiss,In utero and lactational exposure to fluoxetin...,"AH dos Santos, ML Vieira, N de Azevedo Camin, ...",2016,13
7,Ana_Carolina_Kiss,Efeito do extrato aquoso de Allium sativum L. ...,"A Kiss, M Takaku, DC Damasceno, KE Campos, YK ...",2006,12
8,Ana_Carolina_Kiss,Neonatally induced mild diabetes: influence on...,"ACI Kiss, B Woodside, YK Sinzato, MM Bernardi,...",2013,11
9,Ana_Carolina_Kiss,The exposure to Trichilia catigua (catuaba) cr...,"AH dos Santos, AC Ramos, KM Silveira, ACI Kiss...",2015,6


In [152]:
df_profs.groupby('professor').ano.agg(['sum','mean','min','max']).sort_values('sum', ascending = False)

,sum,mean,min,max
professor,,,,
Ana_Carolina_Kiss,40258,2012.90,2006,2019
Pedro_Augusto_Carlos_Magno_Fernandes,40256,2012.80,2005,2018
Gustavo_Rohenkohl,40253,2012.65,2006,2018
Renata_Guimarães_Moreira,40230,2011.50,2005,2018
Cristiéle_da_Silva_Ribeiro,40226,2011.30,2001,2018
Yoshihisa_Oda,40225,2011.25,2005,2017
André_Frazão_Helene,40194,2009.70,2003,2018
Cristina_Ribeiro,40174,2008.70,1999,2017
Gisele_Akemi_Oda,40173,2008.65,1995,2017
